In [1]:
import gzip
import pickle

In [2]:
with gzip.open("cache/engineers/harmonic_site_symmetries.pkl.gz", "rb") as f:
    data = pickle.load(f)

In [3]:
from preprocess_wychoffs import clasterize_harmonics

In [5]:
distances = clasterize_harmonics(data)

In [134]:
def assign_to_clusters(
    distances: pd.DataFrame):

    remaining_distances = distances.copy().droplevel((0, 1), axis=0)
    assert (remaining_distances.index == np.arange(remaining_distances.shape[0])).all()
    assert (remaining_distances.columns == np.arange(remaining_distances.shape[1])).all()
    mapping = np.empty(distances.shape[0], dtype=int)
    
    while not remaining_distances.empty:
        row, col = np.unravel_index(np.argmin(remaining_distances.values), remaining_distances.shape)
        row_label = remaining_distances.index[row]
        col_label = remaining_distances.columns[col]
        # enum -> cluster
        mapping[row_label] = col_label
        remaining_distances = remaining_distances.drop(row_label, axis=0).drop(col_label, axis=1)
    return pd.Series(mapping, index=distances.index.get_level_values(2))

In [152]:
mapping = distances.groupby(level=["spacegroup_number", "site_symmetries"]).apply(assign_to_clusters)

In [153]:
mapping

spacegroup_number  site_symmetries  sites_enumeration
1                  1                0                    5
2                  -1               0                    0
                                    1                    6
                                    2                    3
                                    3                    7
                                                        ..
230                .3.              0                    5
                   .32              0                    5
                   1                0                    5
                   2..              0                    5
                   2.22             0                    5
Length: 1731, dtype: int64

In [169]:
pd.MultiIndex.from_arrays(
    [mapping.index.get_level_values(0), mapping.index.get_level_values(1), mapping.values],
    names=[mapping.index.names[0], mapping.index.names[1], "cluster-index"]
)

MultiIndex([(  1,     '1', 5),
            (  2,    '-1', 0),
            (  2,    '-1', 6),
            (  2,    '-1', 3),
            (  2,    '-1', 7),
            (  2,    '-1', 2),
            (  2,    '-1', 1),
            (  2,    '-1', 4),
            (  2,    '-1', 5),
            (  2,     '1', 5),
            ...
            (229,  'm.m2', 0),
            (229, 'mm2..', 0),
            (230,  '-4..', 5),
            (230,  '.-3.', 5),
            (230,   '..2', 5),
            (230,   '.3.', 5),
            (230,   '.32', 5),
            (230,     '1', 5),
            (230,   '2..', 5),
            (230,  '2.22', 5)],
           names=['spacegroup_number', 'site_symmetries', 'cluster-index'], length=1731)

In [164]:
mapping_expanded.index.set_levels("cluster_index", level=3)

TypeError: Levels must be list-like

In [ ]:
pd.Series(
    data=mapping.index.get_level_values(2),
    index=
)

sites_enumeration
0    1138
1     416
2      92
3      63
4       7
5       7
6       4
7       4
Name: count, dtype: int64

In [127]:
mapping.value_counts()

0    499
5    257
3    203
2    184
4    179
6    178
1    128
7    103
Name: count, dtype: int64

In [65]:
mapping.shape

(8,)

In [64]:
mapping = np.empty(len(d), dtype=int)